In [3]:
##to do
#troubleshoot why force quits happen (NOAA BUOY SCRIPT)
#generate buoy list if it doesn't exist already
#synchronize pi clock with gps clock
#add in weather forecast details
#configure alarms / alerts (within initial sync)
#Make user settings a separate text file
#Make sure only to pull in weather buoy data from buoy that is reporting wind
#Define streaming as a function and have option to overide internal GPS with preset coordinates
#Add in logic for correcting for temperature


import time  
import sys
from datetime import datetime
from ISStreamer.Streamer import Streamer  
from sensehat_data import sense_data        #custom function
from noaa_buoy_data import noaa_buoy        #custom function
from gps_data import get_gps                #custom function


# --------- User Settings ---------
BUCKET_NAME = "Aurora Dock-Watcher"
BUCKET_KEY = "K6S8MB67WJLS"
ACCESS_KEY = "ist_47mQx4kg8OSYyh1l-k8atYDGJSM2SglH"
MINUTES_BETWEEN_SENSEHAT_READS = .5
SEA_STATE_SPAN_SEC = 10
GPS_FIX_SEC = 10
CURRENT_LAT = 41.77  #default values for Providence, will be updated after first fix
CURRENT_LON = -71.39 #default values for Providence, will be updated after first fix
# --------------------------------

In [ ]:
gps_loc = get_gps(GPS_FIX_SEC)                                      #Number is the timeout for finding a GPS fix
if (gps_loc['Lat'] != None) & (gps_loc['Lon'] != None):
    CURRENT_LAT = gps_loc['Lat']
    CURRENT_LON = gps_loc['Lon']
loc_coord_str = {
    "Location": str(CURRENT_LAT) + "N, " + str(CURRENT_LON*(-1)) + "W"      #Assumes always in NW hemisphere
    }
print(gps_loc)

In [4]:
#Import data from SenseHat
sh_snapshot = sense_data(SEA_STATE_SPAN_SEC)
print(sh_snapshot)

{'Temperature (F)': 94.0, 'Humidity (%)': 20.0, 'Pressure (mb)': 915.0, 'Compass Heading': 22.0, 'Roll (deg)': 2.0, 'Pitch (deg)': -11.0, 'Yaw (deg)': 21.0, 'Roll StDev (deg)': 0.0, 'Pitch StDev (deg)': 0.0, 'Yaw StDev (deg)': 0.0}


In [5]:
#Import data from NOAA buoy
buoy_snapshot = noaa_buoy(CURRENT_LAT, CURRENT_LON)
print(buoy_snapshot)

{'Name': 'PVDR1 (1.0 nm away)', 'Timestamp': 'December 18, 2020 1:30 pm EST', 'Location': '41.786N 71.383W', 'Wind Direction (o)': 340.0, 'Wind Speeds (kts)': 6.0, 'Wind Gusts (kts)': 9.9, 'Pressure (mb)': 1022.8, 'Temperature (F)': 31.1, 'Dew Point (F)': 14.9, 'Visibility (nm)': 5.9}


In [6]:
#Create a summary tag to characterizes this reading
readings = {
    "Boat": 'Boat: ' + datetime.now().strftime("%B %d %Y, %H:%M") + " " + str(datetime.now().strftime("%r"))[9].lower() + "m EST",
    "Buoy": buoy_snapshot['Name'] + ': ' + buoy_snapshot['Timestamp']
    }
print(readings)

{'Boat': 'Boat: December 18 2020, 14:01 pm EST', 'Buoy': 'PVDR1 (1.0 nm away): December 18, 2020 1:30 pm EST'}


In [ ]:
print(sh_snapshot)
print(buoy_snapshot)
print(readings)